In [1]:
import pandas as pd
import numpy as np
import sklearn

Import Data

In [2]:
df = pd.read_csv('ass_5_train.csv')
df_t = pd.read_csv('ass_5_test.csv')

Look at Datasets

In [3]:
df.head(5)


,label,id,visitTime,purchaseTime,hour,C1,C2,C3,C4,C5,...,N4,N5,N6,N7,N8,N9,N10,C10,C11,C12
0,-1,61518,19297675,-1,6,928106152,2541203883,3606287998,306394936,2967512603,...,0,0,0,0,0,0,0,1083418268,3831440054,830706566
1,-1,356604,19320277,-1,12,3382400190,2018710401,1302581498,2308170622,2967512603,...,2,0,0,0,6,0,0,2993954402,2379633508,1284343215
2,-1,603441,19346318,-1,20,2569487005,2018710401,4121075497,2210625817,2967512603,...,0,0,0,0,0,0,0,1601090997,3617851689,4220791646
3,-1,192754,19308171,-1,9,2569487005,2018710401,3531445293,652481754,2967512603,...,0,0,0,0,0,0,0,4114483006,3617851689,4220791646
4,-1,210892,19309442,-1,9,1659166197,2018710401,1047694444,2210625817,2967512603,...,0,0,0,0,0,0,0,1971402551,2196958187,830706566


--> Interpretation of visitTime: Time since last visit

In [4]:
#df.visitTime = pd.to_datetime(df.visitTime, unit ='s')
#df.purchaseTime = pd.to_datetime(df.purchaseTime, unit = 's')

In [5]:
import dtale

In [ ]:
# Inspect if the PurchaseTime was converted well
d = dtale.show(df)
# d.open_browser()

In [ ]:
# Replace NaN again with 0 as Dates have been parsed now


In [ ]:
#Look at all columns
df.columns

In [ ]:
df.info()

Transform C columns into categories

In [ ]:
df[['hour', 'C1', 'C2', 'C3','C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12']] = df[['hour', 'C1', 'C2', 'C3','C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12']].astype('category')

In [ ]:
df.head()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

Create Heatmap to see Multicolinearity
- Pearson Correlation can usually only be used for Regressions, however for a binary classification it is also possible.

In [ ]:
df_corr = df.corr()
plt.figure(figsize=(30,25))
sns.heatmap(df_corr, annot = True)

When looking at the Heatmap, as a first step one has to remind itself of the definiton for multicolinearity:
- Multicolinearity is a problem if it is between dependent variables. Hence, when looking at the heatmap one can argue N4 is multicolinear with N8 and therefore once of these variables have to be dropped.

This means, that the high correlation of purchaseTime and visitTime with label does not account for a multicolinear problem

As a next step, one has to decide with which features we want to continue. There are several ways which I will try to different ones:
- Option 1: Only use purchaseTime and visit Time as there very highly correlated with label and therefore promise a good results
- Option 2: Use all the features except for the ones we have to drop due to multicolinearity

In [ ]:
#Select only bottom of the correlation matrix
df_corr_MC = df_corr[df_corr > 0.7]

mask_ut=np.triu(np.ones(df_corr_MC.shape)).astype(np.bool)
plt.figure(figsize=(30,25))
sns.heatmap(df_corr_MC, mask = mask_ut, annot = True)

- Removal of N10 and N4 as they have a correlation with an other independent Variable (multicolinearity) of the chose value 0.7
- N10 and N4 are chosen over N8 and N9 as they have a better correlation with label

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_1 = df[['purchaseTime', 'visitTime']]
x_1.head()

In [ ]:
y = df[['label']]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_1, y, test_size= 0.2)

Creating Option 1:

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
gnb = GaussianNB()
y_pred = gnb.fit(x_train, y_train.values.ravel()).predict(x_test)

Score model

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_test,y_pred))

From the Naive Bayes we see that precision, recall as well as F1 Scores are 1 --> Hence, the model can very accurately predict the label with the features:
-  Option 1:
   - visitTime
   - purchaseTime

Hence, it does not make sense to further investigate other models if the score of the test set will yield in the sames scores

# Trained model used on test DF

In [ ]:
df_t.head()

In [ ]:
df_t.info()

In [ ]:
x_2 = df_t[['purchaseTime', 'visitTime']]
x_2.head()

In [ ]:
y_2 = df_t[['label']]

In [ ]:
y_pred2 = gnb.predict(x_2)

Score model for test df

In [ ]:
print(classification_report(y_2,y_pred2))

Looking at the output of the classification report one could think this id odd.
However, when looking at the labeled data of the test df, one realizes that only -1 is supposed to be predicted. Hence, the NB Algorithm works very accurately with only the two features:
 - visitTime
 - purchaseTime

Hence, Option 2 and further feature selection is not necessary

Preparing deliverable

In [ ]:
y_proba = gnb.predict_proba(x_2)

In [ ]:
y_proba = pd.DataFrame(y_proba)
y_proba

In [ ]:
prob = y_proba[1]
prob

In [ ]:
df_final = pd.DataFrame(prob)
df_final

In [ ]:
df_final = pd.concat([df_t.id, df_final], axis = 1)

In [ ]:
hand_in = df_final.rename(columns={1:'Prob'})

In [ ]:
hand_in

In [ ]:
hand_in.to_csv('Assignment 3&4 Hand in.csv', index = False)




# Out of Curiosity trying with XG Boost
--> Outcome the same as with GNB

Trying with XGboost

In [ ]:
from xgboost import XGBClassifier

In [ ]:
model_x = XGBClassifier()

In [ ]:
model_x.fit(x_train, y_train.values.ravel())
y_pred_x = model_x.predict(x_test)

In [ ]:
print(classification_report(y_test,y_pred_x))

In [ ]:
y_pred_x2 = model_x.predict(x_2)

In [ ]:
print(classification_report(y_2,y_pred_x2))

In [ ]:
y_proba_x = gnb.predict_proba(x_test2)

In [ ]:
pd.DataFrame(y_proba_x).head()